In [1]:
# Inspiration on reading in Chunks from: https://pythonspeed.com/articles/chunking-pandas/
# and https://towardsdatascience.com/why-and-how-to-use-pandas-with-large-data-9594dda2ea4c

import pandas as pd

FILE_PATH = 'in/dependencies-1.6.0-2020-01-12.csv'

df_chunk = pd.read_csv(FILE_PATH, chunksize=100000) #nrows=2

chunk_list = [] 


def _apply_filer(df, target):
    df = df[['Platform','Project Name', 'Dependency Name', 'Dependency Kind', 'Version Number']]
    return df[df['Platform'] == target]

def chunks_df(platform):
    for chunk in df_chunk:  
        chunk_filter = _apply_filer(chunk, platform)
        chunk_list.append(chunk_filter)
        
    df_concat = pd.concat(chunk_list)
    df_concat.to_csv(f'out/dep-libio-{platform}.csv', index=False)

chunks_df('Cargo')


C:\Users\joaki\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [115]:
df_chunk2 = pd.read_csv(FILE_PATH, nrows=2) #nrows=2
df_chunk2


,ID,Platform,Project Name,Project ID,Version Number,Version ID,Dependency Name,Dependency Platform,Dependency Kind,Optional Dependency,Dependency Requirements,Dependency Project ID
0,8737,Dub,a4g,197,0.1.0,2,a4g:client,Dub,runtime,False,*,NaN
1,8738,Dub,a4g,197,0.1.0,2,a4g:server,Dub,runtime,False,*,NaN


In [22]:
import pandas

result = None
for chunk in pandas.read_csv(FILE_PATH, chunksize=1000):
    voters_street = chunk[
        "Platform"]
    chunk_result = voters_street.value_counts()
    if result is None:
        result = chunk_result
    else:
        result = result.add(chunk_result, fill_value=0)

result.sort_values(ascending=False, inplace=True)
print(result) # adds up to the total 194m rows

NPM          154914774.0
Packagist      8987304.0
Maven          7324121.0
NuGet          6596591.0
Rubygems       5732633.0
CPAN           2421659.0
Pypi           2018342.0
Cargo          1027098.0
CRAN            520065.0
Pub             326516.0
Atom            285882.0
Hex              98110.0
Puppet           77659.0
Dub              21455.0
Elm              19130.0
Haxelib           7596.0
Homebrew          5348.0
Conda             4279.0
Name: Platform, dtype: float64


In [2]:
import pandas as pd

ECOSYS = 'Cargo'
print(f"Checking {ECOSYS}")

def _get_frames(target):
    ltarget = target.lower()
    df = pd.read_csv(f"in/{ltarget}_dependencies_05-17-2022.csv")
    df_libio = pd.read_csv(f'out/dep-libio-{target}.csv')
    return df, df_libio

    
df, df_libio = _get_frames(ECOSYS) #Enter case sensitive Libraries io naming
print(f"Total dependencies DaSEA : {len(df)}, Libraries io: {len(df_libio)}")

# Get packages that we share to only look at those dependencies
def _get_shared_packages_sample(target):
    packages_df = pd.read_csv(f'in/{target}_packages_05-17-2022.csv')
    packages_libio_df = pd.read_csv('in/projects-1.6.0-2020-01-12.csv')
    names_dasea = packages_df['name'].str.lower()
    names_libio = packages_libio_df[packages_libio_df['Platform'] == f'{target}']['Name'].str.lower()
    res = names_libio[names_libio.isin(names_dasea)]
    print(f"DaSEA and {target} shares {len(res)}") # Cargo 3.4660 = 35635 - 975 (Correct)
    return res

shared_packages = _get_shared_packages_sample(ECOSYS)

Checking Cargo
Total dependencies DaSEA : 4022415, Libraries io: 1027098


<ipython-input-2-bb75fa31474e>:26: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  shared_packages = _get_shared_packages_sample(ECOSYS)


DaSEA and Cargo shares 34660


In [174]:
# Get one sample package of the packcages shared
sample = shared_packages.sample(1)

In [175]:
# Get dependencies for those packages in Libraries
deps_libio = df_libio[df_libio['Project Name'].isin(sample)].sort_values('Dependency Name')
deps_libio.to_csv('out/dep_sample_libio.csv', index=False)

# Get a list of the unique packages that sample package is dependent on
uniq_libio = deps_libio['Dependency Name'].unique()
print(sample.values[0])

print("Libraries:")

# For each unique package, get its dependencies the version and kind for that sample package
for e in uniq_libio:
    # print(f"{e}") # {deps_libio[deps_libio['Dependency Name'] == f'{e}']['Version Number'].to_list()}")
    v_df = deps_libio[deps_libio['Dependency Name'] == f'{e}']
    v_lst = v_df['Version Number'].to_list()
    v_lst.sort(reverse=True)
    info = ""
    counter = 0
    for i in v_lst:
        if counter >= 2:
            continue
        k = v_df[v_df['Version Number'] == f'{i}']['Dependency Kind'].values
        info += f"[v{i} {k[0]}]"
        counter += 1
    print(f"{e}: {info}")


deps_libio[deps_libio['Dependency Name'] == 'hyper']['Version Number'].to_list() #['Version Number', 'Dependency Name']

deps_dasea = df[df['source_name'].isin(sample)].sort_values('target_name')
deps_dasea.to_csv('out/dep_sample_dasea.csv', index=False)
uniq_dasea = deps_dasea['target_name'].unique()

print()

print("DaSEA:")
for e in uniq_dasea:
    # print(f"{e}") #: {deps_dasea[deps_dasea['target_name'] == f'{e}']['source_version'].to_list()}")

    v_df = deps_dasea[deps_dasea['target_name'] == f'{e}']
    v_lst = v_df['source_version'].to_list()
    v_lst.sort(reverse=True)
    info = ""
    counter = 0
    for i in v_lst:
        if counter >= 2:
            continue
        k = v_df[v_df['source_version'] == f'{i}']['kind'].values
        vv = v_df[v_df['source_version'] == f'{i}']['target_version'].values
        info += f"[v{i} {vv[0]} {k[-1]}] " #i + " (" + k[0] + ") " + " "
        counter += 1
    print(f"{e}: {info}")


graphy_static_util
Libraries:
graphy_error: [v0.2.0 runtime]

DaSEA:
graphy_error: [v0.2.0 ^0.2 NORMAL] 


In [176]:
def _intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

def _symmetric_difference(lst1, lst2):
    for x in lst1:
        if x not in lst2:
            print(x)

dl_intr = _intersection(uniq_libio, uniq_dasea)

print(f"In total they share dependencies between {len(dl_intr)} packages")

for e in dl_intr:
    print(e)

print()
print(f"DaSEA has dependency not in libraries:")
_symmetric_difference(uniq_dasea, uniq_libio)


print()
print(f"Libraries has dependency not in DaSEA:")
_symmetric_difference(uniq_libio, uniq_dasea)

In total they share dependencies between 1 packages
graphy_error

DaSEA has dependency not in libraries:

Libraries has dependency not in DaSEA:
